In [2]:
import evoVAE.utils.seq_tools as st
from evoVAE.utils.datasets import MSA_Dataset
from evoVAE.models.seqVAETest import SeqVAETest
import evoVAE.utils.statistics as stats
import pandas as pd
import torch


pd.set_option("display.max_rows", None)

(14276, 4)


In [3]:
test_aln = st.read_aln_file("../data/pair_test.aln")
#test_aln = pd.read_pickle("../data/gb1/gb1_ancestors_extants_encoded_weighted_no_dupes.pkl")
test_aln.head()


Reading the alignment: ../data/pair_test.aln
Checking for bad characters: ['B', 'J', 'X', 'Z']
Performing one hot encoding
Number of seqs: 8


,id,sequence,encoding
0,seq_1,LTRAALYEDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,seq_2,LTRATLYEDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seq_3,LTRCTLPEDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seq_4,LRRATLPDDC,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,seq_5,LRRATLPDDA,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [4]:

train_dataset = MSA_Dataset(test_aln["encoding"], test_aln["weights"], test_aln["id"])
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=8, shuffle=False
)

SEQ_LEN = 0
BATCH_ZERO = 0
SEQ_ZERO = 0
seq_len = train_dataset[BATCH_ZERO][SEQ_ZERO].shape[SEQ_LEN]
input_dims = seq_len * 21


config={
            # Dataset info
            "alignment": "tets",
            "seq_theta": 0.2,  # reweighting
            "AA_count": 21,  # standard AA + gap
            "test_split": 0.2,
            "max_mutation": 4,  # how many mutations the model will test up to
            # ADAM
            "learning_rate": 1e-2,  # ADAM
            "weight_decay": 1e-4,  # ADAM
            # Hidden units
            "momentum": None,
            "dropout": None,
            # Training loop
            "epochs": 500,
            "batch_size": 128,
            "max_norm": 10,  # gradient clipping
            "patience": 3,
            # Model info - default settings
            "architecture": f"SeqVAE_0.25_ancestors_R",
            "latent_dims": 2,
            "hidden_dims": [256, 128, 64],
            # DMS data
            "dms_file": "../data/SPG1_STRSG_Wu_2016.pkl",
            "dms_metadata": "../data/DMS_substitutions.csv",
            "dms_id": "SPG1_STRSG_Wu_2016",
}



SEQ_LEN = 0
BATCH_ZERO = 0
SEQ_ZERO = 0
seq_len = train_dataset[BATCH_ZERO][SEQ_ZERO].shape[SEQ_LEN]
input_dims = seq_len * 21


good_weights = "../data/gfp/model_weights/ancestors_extants_no_duplicates_gfp_model_state.pt"
gb1_good = "../data/gb1/model_weights/gb1_ancestors_extants_no_dupes_model_state.pt"
bad_weights = "../data/gfp/model_weights/bad_loss/gfp_ancestors_extants_no_duplicates_model_state.pt"
model = SeqVAETest(input_dims, 2, hidden_dims=config["hidden_dims"], config=config)
model.load_state_dict(torch.load(gb1_good))
model.eval()

KeyError: 'weights'

In [8]:

names = []
z_vals = []

num_samples = 50
for encoding, weights, name in train_loader:
    print(name)

    encoding = encoding.float()
    encoding = torch.flatten(encoding, start_dim=1)
    #print(encoding.shape)

    encoding = encoding.expand(num_samples, encoding.shape[0], encoding.shape[1])
    #print(encoding.shape)
    #print(encoding.shape)
    z_mu, z_logvar = model.encode(encoding.float())
    z_samples = model.reparameterise(z_mu, z_logvar)
    #print(z_samples.shape)
    #print(z_samples[:, 0, :])
    mean_z = torch.mean(z_samples, dim=0)
    
    names.extend(name)
    z_vals.extend(mean_z.detach().numpy())


print(len(names))
print(len(z_vals))
id_to_z = pd.DataFrame({"taxa": names, "z": z_vals})
id_to_z.tail()

('UniRef100_UPI0010A1C192/6-384', 'N1868_gb1_tree_14', 'N1770_gb1_tree_24', 'N139_gb1_tree_30', 'N944_gb1_tree_22', 'N2537_gb1_tree_20', 'N1117_gb1_tree_28', 'N2063_gb1_tree_13')
('N1652_gb1_tree_29', 'N2748_gb1_tree_7', 'N1759_gb1_tree_14', 'N1921_gb1_tree_14', 'N957_gb1_tree_7', 'UniRef100_UPI00109C18DB/5-429', 'N918_gb1_tree_20', 'N762_gb1_tree_14')
('N434_gb1_tree_20', 'N2847_gb1_tree_2', 'N2419_gb1_tree_7', 'N19_gb1_tree_29', 'N2823_gb1_tree_2', 'N2774_gb1_tree_14', 'N983_gb1_tree_11', 'N733_gb1_tree_14')
('N401_gb1_tree_8', 'N636_gb1_tree_28', 'N2878_gb1_tree_20', 'N116_gb1_tree_2', 'UniRef100_UPI00067BC0BC/5-479', 'N352_gb1_tree_21', 'N2581_gb1_tree_30', 'N979_gb1_tree_8')
('N990_gb1_tree_28', 'N741_gb1_tree_30', 'N2160_gb1_tree_14', 'N2886_gb1_tree_20', 'N1083_gb1_tree_13', 'N358_gb1_tree_13', 'N1644_gb1_tree_29', 'N1323_gb1_tree_21')
('N1863_gb1_tree_14', 'N1623_gb1_tree_11', 'N214_gb1_tree_1', 'N439_gb1_tree_1', 'N411_gb1_tree_20', 'N673_gb1_tree_7', 'N2094_gb1_tree_30', 'N87

,taxa,z
2995,UniRef100_UPI00076570AE/12-406,"[0.01595139, 0.6036473]"
2996,N348_gb1_tree_22,"[0.10837912, -0.13592747]"
2997,N2880_gb1_tree_1,"[0.08678953, -0.36853698]"
2998,UniRef100_UPI00109D3718/5-387,"[-0.0070025064, -0.20622925]"
2999,N2543_gb1_tree_14,"[-0.100924686, 2.5779505]"


In [9]:
import torch.nn.functional as F
import evoVAE.utils.seq_tools as st
import evoVAE.utils.metrics as mt

count = 0
inputs = []
recons = []
ids = []
# EVALUATE DIFFERENCES BETWEEN THE RECONSTRUCTIONS AND INPUT 
for id, z in zip(id_to_z['taxa'], id_to_z['z']):

    ids.append(id)
    # decode the Z sample and get it into a PPM shape 
    x_hat = model.decode(torch.tensor(z))
    x_hat = x_hat.unsqueeze(-1)
    #print(x_hat.shape)

    x = test_aln[test_aln['id'] == id]['sequence'].values[0]
    x_one_hot = torch.tensor(st.seq_to_one_hot(x))
    #print(x_one_hot.shape)

    orig_shape = tuple(x_one_hot.shape)
    x_hat = x_hat.view(orig_shape)

    indices = x_hat.max(dim=-1).indices.tolist()
    
    orig = test_aln[test_aln['id'] == id]['sequence'].values[0]
    inputs.append(orig)
    recon = "".join([st.GAPPY_PROTEIN_ALPHABET[x] for x in indices])
    recons.append(recon)

    #hamming_dist += mt.hamming_distance(orig, recon)
    #count += 1

print(len(recons))
print(len(inputs))

3000
3000


In [ ]:
print(ids)

In [10]:
recons_df = pd.DataFrame({"id": ids, "sequence": recons})
recon_msa, _, _ = st.convert_msa_numpy_array(recons_df)
#recon_pairs = pair_wise_covariances(recon_msa)

Sequence weight numpy array created with shape (num_seqs, columns):  (3000, 448)


In [12]:
recons_df.head()

,id,sequence
0,UniRef100_UPI0010A1C192/6-384,----TNKNYSLRKLKTGTASVAVAVALGAGFANQTEVKAESPKVSV...
1,N1868_gb1_tree_14,----------------------------------------------...
2,N1770_gb1_tree_24,-KSERKVHYSIRKFSVGVASVVVASLFMGSVVHATEKEGSTQVATS...
3,N139_gb1_tree_30,----TNKNYSLRKLKTGTASVAVAVALGAGFANQTEVKAESPKVSV...
4,N944_gb1_tree_22,--------------------VLAGLLSGAGLVRADSVKAEEKEIEQ...


In [13]:
test_aln.head()

,id,sequence,encoding,weights
43316,UniRef100_UPI0010A1C192/6-384,----TNKQYSLRKLKTGTASVAVALTLGAGFANQTEVKAEGVSVGS...,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.018868
20360,N1868_gb1_tree_14,----------------------ASVALGAGVTSQPTVRAEEAPVAS...,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.013333
32590,N1770_gb1_tree_24,-KSERKVHYSIRKFSIGVASVVVASLFMGSVVHATENEGSTQAATS...,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.041667
40205,N139_gb1_tree_30,----TKKNYSLRKLKTGTASVAVALTLGAGFANQTEVKAEGASSTS...,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.003831
28682,N944_gb1_tree_22,--------------------ISAGLLSGQGVVRAGSVRSEAQLAEQ...,"[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",0.029412


In [14]:
for x, y in zip(test_aln['id'], recons_df['id']):
    print(x == y)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [32]:
for x, y in zip(test_aln['sequence'].values[-5:], recons_df['sequence'].values[-5:]):
    #print(x)
    #print()
    print(y)
    #print()
    print()


----------------------ASVALGAGVASQPTVRAEEAPV-------------------ASQSKAEKDYDAAMKKSEAAKKAYEEAKKDLEEAKAAQKKYEDDQKKTEEKAAAVKKIDEEHQAANLKSQQALVEFLAAQREKKAAQQKLEEAEKAEKEKKKEFDKAQAVVVPEELKETKEKAK--EDEAKAKEEAKKKPEEAKKSEEAEKKAEEAEQKHAKEVVPPAKKVALQANIVQKLEKQ-LKEQDLSDSEDYVKEGLPLQSELDAKKAKLSKLEELSDKID-ELDAEIAKLEVQLKGNNNVEAYFKEGLEKTTAEKKAELEKAEADLKKAVDEPETPAPAPAPAPAPAPTPEPAPAPAAPAPAPAPKPAPAPKPAPAPKPAPAPAPAPAPKPETPKTPAP----------------------------------------

--------------------V-A-L----GL-----VK-EE-EIEQ-KELKNKLKKEREKLKQKEKLKEKKDLKEATEEKQKLKEELEKLKKELKELKQENEELETELEELEEELEKLQQEKAQKLKQELEKTTEQLKELEQQLKEQTEQAKLEEKQEKLKEQLTQNQEEKAQLE-AELEEAKEQLKEEQEAKQASEKQKKLTEKQKTVEKLTAEKEELAAQKAKLEAAKQELAELKTEEQAKLEAKKQLAAEQQKEAEASKQKLDLEAKAEEKQKLLKDQAELSEKEKAELEEELADLEAKLEKLKKELENQKKLSTEEKAKLQKKLEALPQKLKEQEEKPKEPKAPQTPEKKPEEKPETPKTPEKPEKSVPWTPLTPATPETTPKAKPAPKPAPAAKAPTAQAPAANTANQLPQTGEAANPFFTAAALSVMASAGVLATKRKK---

-KSERKVHYSIRKFSIGVASVVVASLFLGGVVHAEEVTG-T---TSSSKVRKSLEKDRHTQDLIKKLQDIKRTYLYNLKEKPEAELTSKTKKELDEKFKK

In [15]:


from evoVAE.utils.seq_tools import AA_TO_IDX, GAPPY_ALPHABET_LEN
import numpy as np

def pair_wise_covariances(msa):

    SEQ_COUNT = 0
    COLS = 1

    pairs = []
    for i in range(st.GAPPY_ALPHABET_LEN):
        pairs.extend([(i, j) for j in range(i + 1, st.GAPPY_ALPHABET_LEN)])

    # the number of unique ways we can compare columns in the MSA 
    column_combinations = (msa.shape[COLS] * (msa.shape[COLS] - 1) // 2)
    # number of different residue combinations we can have 
    aa_combinations = GAPPY_ALPHABET_LEN ** 2

    num_seqs = msa.shape[SEQ_COUNT]
    num_columns = msa.shape[COLS]

    # each column has aa_combinations many ways to combine residues
    # this is an upper triangular matrix but we will store it in a linear format. 
    covariances = np.zeros(column_combinations * aa_combinations)

    # keep track of which column combination we're up to 
    col_combination_count = 0
    for i in range(num_columns - 1):
        for j in range(i + 1, num_columns):
            col_i = msa[:, i]
            col_j = msa[:, j]

            for a, b in pairs:
      
                # find how many sequences have residues a and b
                col_i_res = np.where(col_i == a)[0]
                col_j_res = np.where(col_j == b)[0]

                # find how many sequences have this combination 
                intersect = np.intersect1d(col_i_res, col_j_res, assume_unique=True).shape[SEQ_COUNT]
                # make a frequency based on number of sequences 
                freq_Ai_Bj = intersect / num_seqs
                
                # just count how many sequences have these residues 
                freq_Ai = col_i_res.shape[0] / num_seqs
                freq_Bj = col_j_res.shape[0] / num_seqs

                # get correct position: (which column combination we're at) + (which residue combination we're at)
                covar_index = col_combination_count * aa_combinations + a * st.GAPPY_ALPHABET_LEN + b

                # useful in case you want to find a specific cov score based on column and residue indices in the upper tri matrix
                #col_combination_count = (num_cols*(num_cols-1)/2) - (num_cols-col_1_idx)*((num_cols-col_1_idx)-1)/2 + col_2_idx - col_1_idx - 1
                #covar_index = int(col_combination_count * aa_combinations + a_idx * st.GAPPY_ALPHABET_LEN + b_idx)

                # (joint occurances of residues a & b at thi) - (occurence of A at col i * occurence of B at col j)
                covariances[covar_index] = freq_Ai_Bj - (freq_Ai * freq_Bj)

            # keep track of how many column combinations we've seen 
            col_combination_count += 1

    
    return covariances

##msa, _, _ = st.convert_msa_numpy_array(test_aln)
#pairs = pair_wise_covariances(msa)
#print(pairs.shape)
#pairs.shape

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

gb1_corv_data = pd.read_csv("../data/gb1/covar/ancestors_extants_gb1_msa_cov.csv")
gb1_corv_data.head()


plt.scatter(gb1_corv_data['actual'], gb1_corv_data['prediction'])
plt.xlabel("MSA covariance")
plt.ylabel("VAE covariance")
slope, intercept = np.polyfit(gb1_corv_data['actual'], gb1_corv_data['prediction'], 1)
x = np.arange(min(gb1_corv_data['actual']),max(gb1_corv_data['actual']) + 0.1, 0.1)
regression_line = slope * x + intercept

# Calculate correlation coefficient
correlation_coefficient = np.corrcoef(gb1_corv_data['actual'], gb1_corv_data['prediction'])[0, 1]

# Display correlation value
plt.text(plt.xlim()[0], plt.ylim()[1], f'Pearson\'s correlation: {correlation_coefficient:.2f}', va='top')

# Plot regression line
plt.plot(x, regression_line, color='red')
plt.title("GB1 3000 random extants/ancestors")
plt.show()